### Import Packages

In [1]:
import re
import os
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import pickle

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pandas as pd
from fuzzywuzzy import fuzz
from thefuzz import process
import rapidfuzz

In [2]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shilton.salindeho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/shilton.salindeho/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
ls

EDA all.sql               NLP Wrangling.ipynb       recurring, Q3 to max.csv
EDA.ipynb                 all all.csv               recurring, all.csv
EDA.sql                   check.csv                 results_fuzzymatched.csv
Get Gov Data.sql          merged_results.csv        scratch.sql


### SQL Setup

In [4]:
# Get SQL query file as data input 
def get_sql_file_as_text(file_path):
  
  with open(file_path, 'r') as f:
    sql_content = f.read()
  return sql_content

In [5]:
# Establish connection with Snowflake
conn = snowflake.connector.connect(connection_name="fundingsociety.ap-southeast-1.privatelink")

### Query

In [6]:
# Import SQL Snowflake Query file here
# sql_file = "EDA.sql"

# Get all data instead of just P12M
sql_file = "EDA all.sql"
my_query = get_sql_file_as_text(sql_file)

In [7]:
# Get data from query
results = conn.cursor().execute(my_query).fetch_pandas_all()

In [8]:
# Check data sample
results.head()

,PAYMENT_ID,USER_ID,PAYMENT_DATE,PAYEE_NAME,SCHEDULE_TYPE,PAYEE_COUNTRY,GTV,NET_REVENUE,REVENUE
0,1960a2e9aa28ab27a622b1b0214a31f0,14059,2019-08-21,20341_Breakpoint LLC Invoicing only,one off,SG,4551.341732,21.628998,131.988455
1,5b6b5d70e5953e0eec03af8b711cc572,20337,2019-08-27,20037_Benhills Marketing And Services,one off,SG,758.380000,3.230699,19.717880
2,e2e79344c1541350e7b881d72b5f8763,55,2019-08-20,20825_Yan Ballet East Pte. Ltd.,one off,SG,771.272460,3.291369,20.051567
3,c005dd005af4a2a2cb170f720fcbc9a6,124800,2024-02-06,119169_Novel Haven Pte Ltd,recurring,SG,265.433000,1.296830,4.914302
4,49916d63ec6a551ff566c36c92450b80,14692,2023-11-21,142076_CREV Consultancy Pte Ltd,recurring,SG,2123.464000,26.975577,55.210064


In [9]:
# Check size
results.shape

(18231, 9)

### Clean Payee Names

In [10]:
# Get stop words
stop_words = set(stopwords.words('english'))

# Remove strings up until underscore
results['PAYEE_NAME_EDITED'] = results['PAYEE_NAME'].str.split('_').str[1]

# Lowercase
results['PAYEE_NAME_EDITED'] = results['PAYEE_NAME_EDITED'].str.lower()

# Remove pte, ltd, llc
results['PAYEE_NAME_EDITED'] = results['PAYEE_NAME_EDITED'].str.replace('pte', '').str.replace('ltd', '').str.replace('llc', '').str.replace('llp', '').str.replace('sdn', '').str.replace('bhd', '')

# Tokenize
results['PAYEE_NAME_EDITED_TOKENS'] = results['PAYEE_NAME_EDITED'].apply(lambda x: word_tokenize(x.lower()))
results['PAYEE_NAME_EDITED_TOKENS'] = results['PAYEE_NAME_EDITED_TOKENS'].apply(lambda x: [word for word in x if word not in stop_words])
results['PAYEE_NAME_EDITED_TOKENS'] = results['PAYEE_NAME_EDITED_TOKENS'].apply(lambda x: [token for token in x if re.match(r'\w+', token)])
results['PAYEE_NAME_EDITED_TOKENS_CORPUS'] = results['PAYEE_NAME_EDITED_TOKENS'].apply(lambda x: ' '.join(x))

In [11]:
results.head()

,PAYMENT_ID,USER_ID,PAYMENT_DATE,PAYEE_NAME,SCHEDULE_TYPE,PAYEE_COUNTRY,GTV,NET_REVENUE,REVENUE,PAYEE_NAME_EDITED,PAYEE_NAME_EDITED_TOKENS,PAYEE_NAME_EDITED_TOKENS_CORPUS
0,1960a2e9aa28ab27a622b1b0214a31f0,14059,2019-08-21,20341_Breakpoint LLC Invoicing only,one off,SG,4551.341732,21.628998,131.988455,breakpoint invoicing only,"[breakpoint, invoicing]",breakpoint invoicing
1,5b6b5d70e5953e0eec03af8b711cc572,20337,2019-08-27,20037_Benhills Marketing And Services,one off,SG,758.380000,3.230699,19.717880,benhills marketing and services,"[benhills, marketing, services]",benhills marketing services
2,e2e79344c1541350e7b881d72b5f8763,55,2019-08-20,20825_Yan Ballet East Pte. Ltd.,one off,SG,771.272460,3.291369,20.051567,yan ballet east . .,"[yan, ballet, east]",yan ballet east
3,c005dd005af4a2a2cb170f720fcbc9a6,124800,2024-02-06,119169_Novel Haven Pte Ltd,recurring,SG,265.433000,1.296830,4.914302,novel haven,[novel],novel
4,49916d63ec6a551ff566c36c92450b80,14692,2023-11-21,142076_CREV Consultancy Pte Ltd,recurring,SG,2123.464000,26.975577,55.210064,crev consultancy,"[crev, consultancy]",crev consultancy


### Get Gov Entities Data

In [12]:
# Import SQL Snowflake Query file here
sql_file = "Get Gov Data.sql"
my_query = get_sql_file_as_text(sql_file)

# Get data from query
govdata = conn.cursor().execute(my_query).fetch_pandas_all()

# Head
govdata.head()

,UEN,ISSUANCE_AGENCY_ID,UEN_STATUS,ENTITY_NAME,ENTITY_TYPE,UEN_ISSUE_DATE,REG_STREET_NAME,REG_POSTAL_CODE,UEN,PRIMARY_SSIC_CODE,PRIMARY_SSIC_DESCRIPTION,PRIMARY_USER_DESCRIBED_ACTIVITY,SECONDARY_SSIC_CODE,SECONDARY_SSIC_DESCRIPTION,SECONDARY_USER_DESCRIBED_ACTIVITY
0,53258701D,ACRA,R,SHARON NG MANAGEMENT SERVICES,BN,2014-04-09,YISHUN STREET 11,760122,53258701D,69201,ACCOUNTING AND AUDITING SERVICES (EXCLUDING ON...,na,na,na,na
1,53258705K,ACRA,R,S.ROBIN TRADING,BN,2014-04-09,KEPPEL ROAD,89066,53258705K,52292,FREIGHT TRANSPORT ARRANGEMENT,na,46411,WHOLESALE OF TEXTILES AND LEATHERS,na
2,53258969J,ACRA,R,SQK CONSULTANTS,BN,2014-04-12,SUNSET AVENUE,288244,53258969J,70201,MANAGEMENT CONSULTANCY SERVICES,na,na,na,na
3,53258973D,ACRA,R,SCENIC KUKUP TRAVEL,BN,2014-04-12,SUNRISE TERRACE,805395,53258973D,79102,TRAVEL AGENCIES AND TOUR OPERATORS (MAINLY OUT...,TRAVEL,na,na,TRAVEL
4,53258984W,ACRA,R,SCIENTASK,BN,2014-04-12,ANSON ROAD,79903,53258984W,85509,EDUCATIONAL SUPPORT SERVICES N.E.C. (EXCLUDING...,EDUCATIONAL SUPPORT SERVICES NEC,na,na,EDUCATIONAL SUPPORT SERVICES NEC


In [13]:
govdata.shape

(595017, 15)

In [14]:
# Drop NAN values
govdata.dropna(subset=['ENTITY_NAME'], inplace=True)

# Check size after dropping
govdata.shape

(595017, 15)

In [15]:
# Clean Entity Name

# Lowercase
govdata['ENTITY_NAME_EDITED'] = govdata['ENTITY_NAME'].str.lower()

# Remove pte, ltd, llc
govdata['ENTITY_NAME_EDITED'] = govdata['ENTITY_NAME_EDITED'].str.replace('pte', '').str.replace('ltd', '').str.replace('llc', '').str.replace('llp', '').str.replace('sdn', '').str.replace('bhd', '')

# Tokenize
govdata['ENTITY_NAME_EDITED_TOKENS'] = govdata['ENTITY_NAME_EDITED'].apply(lambda x: word_tokenize(x.lower()))
govdata['ENTITY_NAME_EDITED_TOKENS'] = govdata['ENTITY_NAME_EDITED_TOKENS'].apply(lambda x: [word for word in x if word not in stop_words])
govdata['ENTITY_NAME_EDITED_TOKENS'] = govdata['ENTITY_NAME_EDITED_TOKENS'].apply(lambda x: [token for token in x if re.match(r'\w+', token)])
govdata['ENTITY_NAME_EDITED_TOKENS_CORPUS'] = govdata['ENTITY_NAME_EDITED_TOKENS'].apply(lambda x: ' '.join(x))

In [16]:
govdata.head()

,UEN,ISSUANCE_AGENCY_ID,UEN_STATUS,ENTITY_NAME,ENTITY_TYPE,UEN_ISSUE_DATE,REG_STREET_NAME,REG_POSTAL_CODE,UEN,PRIMARY_SSIC_CODE,PRIMARY_SSIC_DESCRIPTION,PRIMARY_USER_DESCRIBED_ACTIVITY,SECONDARY_SSIC_CODE,SECONDARY_SSIC_DESCRIPTION,SECONDARY_USER_DESCRIBED_ACTIVITY,ENTITY_NAME_EDITED,ENTITY_NAME_EDITED_TOKENS,ENTITY_NAME_EDITED_TOKENS_CORPUS
0,53258701D,ACRA,R,SHARON NG MANAGEMENT SERVICES,BN,2014-04-09,YISHUN STREET 11,760122,53258701D,69201,ACCOUNTING AND AUDITING SERVICES (EXCLUDING ON...,na,na,na,na,sharon ng management services,"[sharon, ng, management, services]",sharon ng management services
1,53258705K,ACRA,R,S.ROBIN TRADING,BN,2014-04-09,KEPPEL ROAD,89066,53258705K,52292,FREIGHT TRANSPORT ARRANGEMENT,na,46411,WHOLESALE OF TEXTILES AND LEATHERS,na,s.robin trading,"[s.robin, trading]",s.robin trading
2,53258969J,ACRA,R,SQK CONSULTANTS,BN,2014-04-12,SUNSET AVENUE,288244,53258969J,70201,MANAGEMENT CONSULTANCY SERVICES,na,na,na,na,sqk consultants,"[sqk, consultants]",sqk consultants
3,53258973D,ACRA,R,SCENIC KUKUP TRAVEL,BN,2014-04-12,SUNRISE TERRACE,805395,53258973D,79102,TRAVEL AGENCIES AND TOUR OPERATORS (MAINLY OUT...,TRAVEL,na,na,TRAVEL,scenic kukup travel,"[scenic, kukup, travel]",scenic kukup travel
4,53258984W,ACRA,R,SCIENTASK,BN,2014-04-12,ANSON ROAD,79903,53258984W,85509,EDUCATIONAL SUPPORT SERVICES N.E.C. (EXCLUDING...,EDUCATIONAL SUPPORT SERVICES NEC,na,na,EDUCATIONAL SUPPORT SERVICES NEC,scientask,[scientask],scientask


### Fuzzy Matching Explore

In [17]:
# Create Fuzzy Match Function
def fuzzy_match(x, choices, cutoff=90):
    return rapidfuzz.process.extractOne(x, choices, scorer=rapidfuzz.fuzz.token_sort_ratio)

In [18]:
# Get unique values of entity names, make into new df
results_payeenameonly = results[['PAYEE_COUNTRY','PAYEE_NAME_EDITED_TOKENS_CORPUS']].copy()
results_payeenameonly = results_payeenameonly.drop_duplicates()
results_payeenameonly.head()

,PAYEE_COUNTRY,PAYEE_NAME_EDITED_TOKENS_CORPUS
0,SG,breakpoint invoicing
1,SG,benhills marketing services
2,SG,yan ballet east
3,SG,novel
4,SG,crev consultancy


In [19]:
# Check size
results_payeenameonly.shape

(5377, 2)

In [20]:
# Fuzzy Match payee name in tx data to entity name in gov data

results_payeenameonly['PAYEE_NAME_MATCHING_TUPLE'] = results_payeenameonly['PAYEE_NAME_EDITED_TOKENS_CORPUS'].apply(
    lambda x: fuzzy_match(x, govdata['ENTITY_NAME_EDITED_TOKENS_CORPUS'].tolist(), cutoff=90))

results_payeenameonly['PAYEE_NAME_MATCHING'] = results_payeenameonly['PAYEE_NAME_MATCHING_TUPLE'].apply(lambda x: x[0])

results_payeenameonly['PAYEE_NAME_MATCHING_SCORE'] = results_payeenameonly['PAYEE_NAME_MATCHING_TUPLE'].apply(lambda x: x[1])

In [21]:
# Perform the left join
results_payeenameonly = results_payeenameonly.merge(
    govdata[['ENTITY_NAME_EDITED_TOKENS_CORPUS', 'PRIMARY_SSIC_DESCRIPTION', 'PRIMARY_USER_DESCRIBED_ACTIVITY', 'SECONDARY_SSIC_DESCRIPTION', 'SECONDARY_USER_DESCRIBED_ACTIVITY']],
    how='left',
    left_on='PAYEE_NAME_MATCHING',
    right_on='ENTITY_NAME_EDITED_TOKENS_CORPUS'
)

# Drop the redundant 'ENTITY_NAME_EDITED_TOKENS_CORPUS' column if not needed
results_payeenameonly.drop(columns=['ENTITY_NAME_EDITED_TOKENS_CORPUS'], inplace=True)

# Get only one unique row for each payee name
results_payeenameonly = results_payeenameonly.drop_duplicates(subset='PAYEE_NAME_EDITED_TOKENS_CORPUS')

In [22]:
results_payeenameonly.shape

(5370, 9)

In [23]:
results_payeenameonly.head()

,PAYEE_COUNTRY,PAYEE_NAME_EDITED_TOKENS_CORPUS,PAYEE_NAME_MATCHING_TUPLE,PAYEE_NAME_MATCHING,PAYEE_NAME_MATCHING_SCORE,PRIMARY_SSIC_DESCRIPTION,PRIMARY_USER_DESCRIBED_ACTIVITY,SECONDARY_SSIC_DESCRIPTION,SECONDARY_USER_DESCRIBED_ACTIVITY
0,SG,breakpoint invoicing,"(breakpoint, 66.66666666666667, 201770)",breakpoint,66.666667,LEGAL ACTIVITIES (EXCLUDING ONLINE MARKETPLACES),PROVISION OF LEGAL SERVICES,na,na
1,SG,benhills marketing services,"(benhills marketing services, 100.0, 213496)",benhills marketing services,100.000000,OTHER BUSINESS SUPPORT SERVICE ACTIVITIES (E.G...,FLYER DISTRIBUTION SERVICES,OTHER SPECIALISED OFFICE SUPPORT ACTIVITIES N....,FLYER DISTRIBUTION SERVICES
2,SG,yan ballet east,"(yan ballet east, 100.0, 247950)",yan ballet east,100.000000,"TRAINING COURSES FOR MUSIC, DANCING, ART, SPEE...",BALLET INSTRUCTION,na,na
3,SG,novel,"(novel, 100.0, 499902)",novel,100.000000,RETAIL SALE OF COMPUTER HARDWARE (INCLUDING HA...,ONLINE SALES OF MOBILE ACCESSORIES AND EREADERS,na,na
4,SG,crev consultancy,"(crev consultancy, 100.0, 389922)",crev consultancy,100.000000,REAL ESTATE AGENCIES AND VALUATION SERVICES,REAL ESTATE AGENCY VALUATION,REAL ESTATE ACTIVITIES ON A FEE OR COMMISSION ...,REAL ESTATE ACTIVITIES ONCONTRACT ON FEE


In [24]:
# Save the results to a CSV file
results_payeenameonly.to_csv('results_fuzzymatched.csv', index=False)

In [25]:
# Load CSV file
results_payeenameonly = pd.read_csv('results_fuzzymatched.csv')

In [26]:
# Recombine with GTV data
merged_results = results.merge(results_payeenameonly.drop(columns=['PAYEE_COUNTRY']), on='PAYEE_NAME_EDITED_TOKENS_CORPUS', how='left')
merged_results.head()

,PAYMENT_ID,USER_ID,PAYMENT_DATE,PAYEE_NAME,SCHEDULE_TYPE,PAYEE_COUNTRY,GTV,NET_REVENUE,REVENUE,PAYEE_NAME_EDITED,PAYEE_NAME_EDITED_TOKENS,PAYEE_NAME_EDITED_TOKENS_CORPUS,PAYEE_NAME_MATCHING_TUPLE,PAYEE_NAME_MATCHING,PAYEE_NAME_MATCHING_SCORE,PRIMARY_SSIC_DESCRIPTION,PRIMARY_USER_DESCRIBED_ACTIVITY,SECONDARY_SSIC_DESCRIPTION,SECONDARY_USER_DESCRIBED_ACTIVITY
0,1960a2e9aa28ab27a622b1b0214a31f0,14059,2019-08-21,20341_Breakpoint LLC Invoicing only,one off,SG,4551.341732,21.628998,131.988455,breakpoint invoicing only,"[breakpoint, invoicing]",breakpoint invoicing,"('breakpoint', 66.66666666666667, 201770)",breakpoint,66.666667,LEGAL ACTIVITIES (EXCLUDING ONLINE MARKETPLACES),PROVISION OF LEGAL SERVICES,na,na
1,5b6b5d70e5953e0eec03af8b711cc572,20337,2019-08-27,20037_Benhills Marketing And Services,one off,SG,758.380000,3.230699,19.717880,benhills marketing and services,"[benhills, marketing, services]",benhills marketing services,"('benhills marketing services', 100.0, 213496)",benhills marketing services,100.000000,OTHER BUSINESS SUPPORT SERVICE ACTIVITIES (E.G...,FLYER DISTRIBUTION SERVICES,OTHER SPECIALISED OFFICE SUPPORT ACTIVITIES N....,FLYER DISTRIBUTION SERVICES
2,e2e79344c1541350e7b881d72b5f8763,55,2019-08-20,20825_Yan Ballet East Pte. Ltd.,one off,SG,771.272460,3.291369,20.051567,yan ballet east . .,"[yan, ballet, east]",yan ballet east,"('yan ballet east', 100.0, 247950)",yan ballet east,100.000000,"TRAINING COURSES FOR MUSIC, DANCING, ART, SPEE...",BALLET INSTRUCTION,na,na
3,c005dd005af4a2a2cb170f720fcbc9a6,124800,2024-02-06,119169_Novel Haven Pte Ltd,recurring,SG,265.433000,1.296830,4.914302,novel haven,[novel],novel,"('novel', 100.0, 499902)",novel,100.000000,RETAIL SALE OF COMPUTER HARDWARE (INCLUDING HA...,ONLINE SALES OF MOBILE ACCESSORIES AND EREADERS,na,na
4,49916d63ec6a551ff566c36c92450b80,14692,2023-11-21,142076_CREV Consultancy Pte Ltd,recurring,SG,2123.464000,26.975577,55.210064,crev consultancy,"[crev, consultancy]",crev consultancy,"('crev consultancy', 100.0, 389922)",crev consultancy,100.000000,REAL ESTATE AGENCIES AND VALUATION SERVICES,REAL ESTATE AGENCY VALUATION,REAL ESTATE ACTIVITIES ON A FEE OR COMMISSION ...,REAL ESTATE ACTIVITIES ONCONTRACT ON FEE


In [27]:
# Save the merged results to a CSV file
merged_results.to_csv('merged_results.csv', index=False)

### Push matched data into Snowflake

In [28]:
# Create table with just payee name, payee name cleaned, entity name, and ssics
merged_results_table = merged_results[['PAYEE_NAME', 'PAYEE_NAME_EDITED_TOKENS_CORPUS', 'PAYEE_NAME_MATCHING', 'PAYEE_NAME_MATCHING_SCORE', 'PRIMARY_SSIC_DESCRIPTION', 'PRIMARY_USER_DESCRIBED_ACTIVITY', 'SECONDARY_SSIC_DESCRIPTION', 'SECONDARY_USER_DESCRIBED_ACTIVITY']].copy()
merged_results_table.drop_duplicates(inplace=True)
merged_results_table.head()

,PAYEE_NAME,PAYEE_NAME_EDITED_TOKENS_CORPUS,PAYEE_NAME_MATCHING,PAYEE_NAME_MATCHING_SCORE,PRIMARY_SSIC_DESCRIPTION,PRIMARY_USER_DESCRIBED_ACTIVITY,SECONDARY_SSIC_DESCRIPTION,SECONDARY_USER_DESCRIBED_ACTIVITY
0,20341_Breakpoint LLC Invoicing only,breakpoint invoicing,breakpoint,66.666667,LEGAL ACTIVITIES (EXCLUDING ONLINE MARKETPLACES),PROVISION OF LEGAL SERVICES,na,na
1,20037_Benhills Marketing And Services,benhills marketing services,benhills marketing services,100.000000,OTHER BUSINESS SUPPORT SERVICE ACTIVITIES (E.G...,FLYER DISTRIBUTION SERVICES,OTHER SPECIALISED OFFICE SUPPORT ACTIVITIES N....,FLYER DISTRIBUTION SERVICES
2,20825_Yan Ballet East Pte. Ltd.,yan ballet east,yan ballet east,100.000000,"TRAINING COURSES FOR MUSIC, DANCING, ART, SPEE...",BALLET INSTRUCTION,na,na
3,119169_Novel Haven Pte Ltd,novel,novel,100.000000,RETAIL SALE OF COMPUTER HARDWARE (INCLUDING HA...,ONLINE SALES OF MOBILE ACCESSORIES AND EREADERS,na,na
4,142076_CREV Consultancy Pte Ltd,crev consultancy,crev consultancy,100.000000,REAL ESTATE AGENCIES AND VALUATION SERVICES,REAL ESTATE AGENCY VALUATION,REAL ESTATE ACTIVITIES ON A FEE OR COMMISSION ...,REAL ESTATE ACTIVITIES ONCONTRACT ON FEE


In [29]:
# Export to SQL for EDA

cursor = conn.cursor()
cursor.execute("""
DROP TABLE IF EXISTS DEV.SBOX_SHILTON.CU_B2C_PAYEE_NAME_MATCHED;
""")

# Commit the changes
conn.commit()

df_export_cluster = merged_results_table

df_export_cluster.columns = df_export_cluster.columns.str.upper()

write_pandas(
    conn,
    df_export_cluster,
    table_name='CU_B2C_PAYEE_NAME_MATCHED',
    database='DEV',
    schema='SBOX_SHILTON',
    auto_create_table=True)

/var/folders/sp/wnd46psj0r5_ys790_zv7xgm0000gr/T/ipykernel_52076/2843146935.py:15: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(


(True,
 1,
 7861,
 [('eqbqyagdsj/file0.txt',
   'LOADED',
   7861,
   7861,
   1,
   0,
   None,
   None,
   None,
   None)])